In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt

## 3.2 Finite difference method

In [2]:
midpoint

NameError: name 'midpoint' is not defined

In [3]:
math.e**4.7

109.9471724521235

In [204]:
M2 = 1
M1 = 123458
np.log(np.linspace(M1, M2, N))
stocks = np.log(np.linspace(M1, M2, N))

In [15]:
sigma = 0.3
K = 110
S = 120
r = 0.04
N = 100
T = 1

midpoint = math.log(S)
M2 = -5
M1 = (midpoint-M2) + midpoint

stocks = np.log(np.linspace(M1, M2, N))

In [16]:
delx = (M1 - M2) / N
deltau = T / N

In [17]:
grid = np.zeros((N,N))


In [18]:
grid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
from scipy.stats import norm
def Black_Scholes_Call(S, K, r, vol, tau):
    """ 
    Black Scholes Model for European Call
    """
    d1 = (np.log(S / K) + (r + ((vol**2)/2.)*tau)) / (vol*np.sqrt(tau))
    d2 = d1 - vol*np.sqrt(tau)
    V = S * norm.cdf(d1) - np.exp(-r*tau) * K * norm.cdf(d2)
    
    return V

In [158]:
M1

14.210340371976184

In [164]:
np.log(S)

4.605170185988092

In [163]:
np.log(S) + np.log(delx*(N/2))

6.886275403655313

In [162]:
np.log(S) - np.log(delx*(N/2))

2.32406496832087

In [205]:
stocks = np.log(np.linspace(M1, M2, N))

In [206]:
# Define matrix A
N = 100
A = np.zeros((N,N))
amin1 = (0.5*sigma**2 * deltau / delx**2)    -    ((r-0.5*sigma**2) *deltau / (2*delx))
aplus1 = (0.5*sigma**2 * deltau / delx**2)    +    ((r-0.5*sigma**2) *deltau / (2*delx))
anow = 1-(sigma**2 * deltau / delx**2) - r * deltau

np.fill_diagonal(A[1:], amin1)
np.fill_diagonal(A[:,1:], aplus1)
np.fill_diagonal(A, anow)
k1, k2, k3, k4 = M1, M1, M2, M2

sigma, K, S, r, N, T = 0.3, 110, 100, 0.04, 100, 1

#midpoint = math.log(S)
#M2 = -5
#M1 = (midpoint-M2) + midpoint

#M1 = np.log(S) + delx*(N/2)
#M2 = np.log(S) - delx*(N/2)

# Stock price at T
#stocks = np.linspace(M1,M2,N) 
print(f"The Correct answer is {Black_Scholes_Call(S, K, r, sigma, 1)}")


# Boundaries
A[0,0] = k1
A[0,1] = k2
A[-1,-2] = k3
A[-1, -1] = k4

The Correct answer is 9.625357828843697


In [207]:
# Get the call prices at maturity
V = np.zeros((N,N))
stocks = np.linspace(M1,M2,N) 

for row in range(V.shape[0]):
    #ans = Black_Scholes_Call(np.exp(stocks[row]), K, r, sigma, T)
    ans = np.exp(stocks[row])*np.exp(-r)
    if ans > 0:
        V[row,0] = ans
    else:
        V[row,0] = 0

C:\Users\sebas\AppData\Local\Temp/ipykernel_7600/2109465226.py:7: RuntimeWarning: overflow encountered in exp
  ans = np.exp(stocks[row])*np.exp(-r)


In [208]:
for column in range(0,N-1):
    V[:,column+1] = A @ V[:,column]

C:\Users\sebas\AppData\Local\Temp/ipykernel_7600/4263752455.py:2: RuntimeWarning: invalid value encountered in matmul
  V[:,column+1] = A @ V[:,column]


In [209]:
V[:,-1][49]

nan

In [210]:
np.exp(stocks[40:55])

C:\Users\sebas\AppData\Local\Temp/ipykernel_7600/3344671722.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(stocks[40:55])


array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf])

In [114]:
len(stocks)

100

In [113]:
np.exp(stocks)

array([2.13714949e+06, 1.75373153e+06, 1.43910115e+06, 1.18091742e+06,
       9.69053462e+05, 7.95199222e+05, 6.52535518e+05, 5.35466573e+05,
       4.39400527e+05, 3.60569330e+05, 2.95880941e+05, 2.42798053e+05,
       1.99238567e+05, 1.63493924e+05, 1.34162094e+05, 1.10092577e+05,
       9.03412812e+04, 7.41334914e+04, 6.08334803e+04, 4.99195742e+04,
       4.09636910e+04, 3.36145491e+04, 2.75838892e+04, 2.26351673e+04,
       1.85742770e+04, 1.52419357e+04, 1.25074373e+04, 1.02635250e+04,
       8.42218454e+03, 6.91119206e+03, 5.67128106e+03, 4.65381784e+03,
       3.81889387e+03, 3.13376047e+03, 2.57154428e+03, 2.11019318e+03,
       1.73161135e+03, 1.42094947e+03, 1.16602226e+03, 9.56830591e+02,
       7.85169210e+02, 6.44304953e+02, 5.28712622e+02, 4.33858277e+02,
       3.56021394e+02, 2.92148934e+02, 2.39735591e+02, 1.96725530e+02,
       1.61431742e+02, 1.32469880e+02, 1.08703956e+02, 8.92017880e+01,
       7.31984304e+01, 6.00661751e+01, 4.92899283e+01, 4.04470075e+01,
      

In [ ]:
V = 

In [9]:
amin1 = (0.5*sigma**2 * deltau / delx**2)    -    ((r-0.5*sigma**2) *deltau / (2*delx))
aplus1 = (0.5*sigma**2 * deltau / delx**2)    +    ((r-0.5*sigma**2) *deltau / (2*delx))
anow = 1-(sigma**2 * deltau / delx**2) - r * deltau

np.fill_diagonal(A[1:], amin1)
np.fill_diagonal(A[:,1:], aplus1)
np.fill_diagonal(A, anow)


k1 = M1
k2 = M1
k3 = M2
k4 = M2

A[0,0] = k1
A[0,1] = k2
A[-1,-2] = k3
A[-1, -1] = k4

In [10]:
for row in range(grid.shape[0]):
    ans = math.e**(-r*T) * stocks[row] - math.log(K)
    if ans > 0:
        grid[row,0] = ans
    else:
        grid[row,0] = 0

In [11]:
for column in range(N-2):
    grid[:,column+1] = A @ grid[:,column]

In [12]:
np.mean(grid[:,-1])

0.0

In [13]:
np.matmul(A, grid[:,0])

array([2.68413560e+02, 9.10943904e+00, 8.91954091e+00, 8.72964279e+00,
       8.53974466e+00, 8.34984654e+00, 8.15994841e+00, 7.97005028e+00,
       7.78015216e+00, 7.59025403e+00, 7.40035591e+00, 7.21045778e+00,
       7.02055966e+00, 6.83066153e+00, 6.64076341e+00, 6.45086528e+00,
       6.26096715e+00, 6.07106903e+00, 5.88117090e+00, 5.69127278e+00,
       5.50137465e+00, 5.31147653e+00, 5.12157840e+00, 4.93168028e+00,
       4.74178215e+00, 4.55188402e+00, 4.36198590e+00, 4.17208777e+00,
       3.98218965e+00, 3.79229152e+00, 3.60239340e+00, 3.41249527e+00,
       3.22259715e+00, 3.03269902e+00, 2.84280090e+00, 2.65290277e+00,
       2.46300464e+00, 2.27310652e+00, 2.08320839e+00, 1.89331027e+00,
       1.70341214e+00, 1.51351402e+00, 1.32361589e+00, 1.13371777e+00,
       9.43819639e-01, 7.53921514e-01, 5.64023388e-01, 3.74125263e-01,
       1.84293602e-01, 2.18735920e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [14]:
grid[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])